In [206]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

ticker = 'TSLA'
url = "https://finance.yahoo.com/quote/TSLA?p=TSLA"
html = requests.get(url)

In [207]:
s = BeautifulSoup(html.content, "html.parser")

In [208]:
results_info = s.find(id = 'Lead-5-QuoteHeader-Proxy')
company_title = results_info.find_all('h1', class_= 'D(ib) Fz(18px)')[0].text
current_price = results_info.find_all('fin-streamer', class_='Fw(b) Fz(36px) Mb(-4px) D(ib)')[0].text

print(company_title)

Tesla, Inc. (TSLA)


In [209]:
financials = s.find(id = 'Main')
current_financials = financials.find_all('td', class_='Ta(end) Fw(600) Lh(14px)')

In [210]:
market_cap = current_financials[8].text
pe_ratio = current_financials[10].text
eps = current_financials[11].text

print(market_cap, pe_ratio, eps)

582.537B 59.11 3.10


In [211]:
#Get tickers of similar companies to user input

similar = s.find('section', id = 'similar-by-symbol')
companys = similar.find_all('a', class_= 'Fw(b) Ell D(b) C($linkColor) Pos(r) Z(2)')

company_similar_ticker = [ticker]
len(companys)

for i in range(len(companys)):
    company_similar_ticker.append(companys[i].text)

company_similar_ticker

['TSLA', 'NIO', 'RIVN', 'LCID', 'F', 'XPEV']

In [224]:
company_name = []
curr_price = []
mar_cap = []
per = []
earnings = []

for tick in company_similar_ticker:
    url_1 = f"https://finance.yahoo.com/quote/{tick}?p={tick}"

    html_1 = requests.get(url_1)
    t = BeautifulSoup(html_1.content, "html.parser")

    results = t.find(id = 'Lead-5-QuoteHeader-Proxy')
    comp_title = results.find_all('h1', class_= 'D(ib) Fz(18px)')[0].text
    cur_price = results.find_all('fin-streamer', class_='Fw(b) Fz(36px) Mb(-4px) D(ib)')[0].text

    financial = t.find(id = 'Main')
    curr_financials = financial.find_all('td', class_='Ta(end) Fw(600) Lh(14px)')   

    market_cap = curr_financials[8].text
    eps = curr_financials[11].text  
    
    company_name.append(comp_title)
    curr_price.append(cur_price)
    mar_cap.append(market_cap)
    earnings.append(eps)


similar_dict = {'Tickers': company_similar_ticker, 'Company Name': company_name, 'Current Price': curr_price, 'Market Cap': mar_cap, 'EPS': earnings}
similar_df = pd.DataFrame(similar_dict)
per = similar_df.get('Current Price').astype(float) / similar_df.get('EPS').astype(float)
similar_df['P/E Ratio']= per
similar_df


    


,Tickers,Company Name,Current Price,Market Cap,EPS,P/E Ratio
0,TSLA,"Tesla, Inc. (TSLA)",183.25,582.537B,3.10,59.112903
1,NIO,NIO Inc. (NIO),6.14,12.771B,-1.79,-3.430168
2,RIVN,"Rivian Automotive, Inc. (RIVN)",15.29,14.644B,-6.02,-2.539867
3,LCID,"Lucid Group, Inc. (LCID)",2.6500,6.067B,-1.5000,-1.766667
4,F,Ford Motor Company (F),11.39,45.594B,1.53,7.444444
5,XPEV,XPeng Inc. (XPEV),8.97,8.462B,-1.62,-5.537037


In [226]:
main_PE = similar_df.loc[0, 'P/E Ratio']
main_EPS = similar_df.loc[0, 'EPS']

print(main_PE, main_EPS)

59.11290322580645 3.10


In [231]:
eps_comparison = main_EPS > similar_df.get('EPS')
if(eps_comparison.sum() > len(eps_comparison) * 0.8):
    print(eps_comparison.sum(), len(eps_comparison) * 0.66)
    indicator = 0.3

if(main_PE > 0.25):
    if(indicator != 0):
        indicator = indicator * 0.5
    else:
        indicator = -0.2
else:
    if(indicator != 0):
        indicator = indicator * 2
    else:
        indicator = 0.2

5 3.96
